In [70]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install yfinance -q

In [57]:
import yfinance as yf

df = yf.download("AAPL", start="2020-01-01", end="2024-03-01").drop(columns=['Adj Close'])
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2024-02-23,185.009995,185.039993,182.229996,182.520004,45119700
2024-02-26,182.240005,182.759995,180.649994,181.160004,40867400
2024-02-27,181.100006,183.919998,179.559998,182.630005,54318900
2024-02-28,182.509995,183.119995,180.130005,181.419998,48953900
2024-02-29,181.270004,182.570007,179.529999,180.750000,136682600


In [58]:
df.shape

(1047, 5)

In [14]:
df.dtypes

Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

In [67]:
df.isnull().sum()

open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [75]:
df.describe()

,open,high,low,close,volume
count,1047.000000,1047.000000,1047.000000,1047.000000,1.047000e+03
mean,142.456982,144.092796,140.923694,142.589016,9.732060e+07
std,33.833835,33.933948,33.708161,33.835379,5.404561e+07
min,57.020000,57.125000,53.152500,56.092499,2.404830e+07
25%,124.630001,126.049999,123.061249,124.816250,6.211980e+07
50%,146.850006,148.500000,145.520004,146.830002,8.277270e+07
75%,169.964996,171.555000,168.180000,170.150002,1.143590e+08
max,198.020004,199.619995,197.000000,198.110001,4.265100e+08


In [72]:
!pip install summarytools -q

In [71]:
from summarytools import dfSummary

dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,open[float64],Mean (sd) : 142.5 (33.8)min < med < max:57.0 < 146.9 < 198.0IQR (CV) : 45.3 (4.2),995 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAClklEQVR4nO3cT27aQBiG8W9KiAxIRggJdlygUhcsOUQP2yN0wQm4ATskhIxwY2GMu2kqRUrB+E89r3l+exMvnsR8k/G4PM8N8N2Xtm8AKOKl7RvAR865gZm9VviIc57nb3Xdjy8I1SPOucF0Ov0ehuGk7Gccj8eDc+5H12IlVL+8hmE4Wa1Wb+PxOHn04iiKgvV6Pdnv969mRqho1ng8Tmaz2a+Slw9qvRlPMExBAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCArun8JfPm7YJFWbm/6ZtQsU7rzdtEyo+8HXTNsMUJBAqJPDob0CF6Tm8Xq/9uu+nCwi1ZlWm5zRNgyzLviZJ8tPMyn5P7CRCrV/p6Xm73U42m83wcrn0mro5VYTakDLT8+Fw6OSrznVgmIIEQoUEHv3/wOTuF0L9BJO7fwj1c7KTe5ZlfTMLnXOPXur1k4BQb1Cb3OM47qdp+m2xWPR6vd5Dv2C+PwkItUPO5/NLEATD5XKZzOfzwyPXtv0kuIdQO2g0Gkk9CYpgeQoSCBUSCBUS+I6K2lRYGjO782IgoaIWVZbGzO6/GEioqEWVpbEiLwYSKmpVZmnsj5vLYwxTkECokECokNDod1SfzzKClsZC9f0sI2hp8i+q12cZQUvjy1O+nmUELQxTkECokECokECokECokECokECokHB3HbXNo22a3DEOLTdDbfNom6Z3jEPLvb+orR1t0/SOcWgp9C/UNo+2aWrHOLQwTEFCZ9+ZqjiIeX2y3TPqZKhVBzHfT7Z7Rp0MtcogZub/yXbPqJOhvis7iPl+st0zYpiCBEKFhEKP/iiKgkc/+HQ6BWZmcRwHu91u+D+vV/3Zqvdd9foiff0Giq2y73H/09gAAAAASUVORK5CYII="">",0(0.0%)
2,high[float64],Mean (sd) : 144.1 (33.9)min < med < max:57.1 < 148.5 < 199.6IQR (CV) : 45.5 (4.2),993 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACmElEQVR4nO3cQW7aQBiG4X9KiAxIthAS7LhApS5YcogetkfoghNwA3ZIyDLCCQIM3TSVoqaAPXaZz3mfvcksXuGZITPucrkYELovjx4AcI+nRw8A7znnemb27PERh8vl8lrXeEJBqAFxzvVGo9H3OI6HVT9ju92mzrkfbYuVUMPyHMfxcD6fvyZJsi/7cJZl0WKxGG42m2czI1Q0K0mS/Xg8fqn4eK/WwQSCxRQkECokECokECokECokECokECokECokECokECokECokECokECokECok8G9++CPk0wWECjML/3QBoeJN0KcLCBXvhHq6gMUUJBAqJBAqJBAqJLCYaoDHfmR8Pp+7dY+nDQi1Zj77kcfjMSqK4ut+v/9pZlVX3q1EqPWrvB+5Wq2Gy+WyfzqdOk0NThWh/oPv6ztJkrTsfmSapt77kEVRdM0sds6VfTToaQehfkD19Z3nefd4PH6bTqedTqdT6ts89GkHoX5M8vV9OByeoijqz2az/WQyScs8G/q0g1CvqPJzYh2vb1+DwUBy3NewjwoJhAoJhAoJhAoJLKZQG489XLMbx1gIFbXw2cM1u32MhVBRC5893HuOsRAqalVlD/e3q/u4LKYggVAhgVAhodE5asg3b0BLY6GGfvMGtDT5jRr0zRvQ0vj2VKg3b0ALiylIIFRIIFRIIFRIIFRIIFRIIFRIIFRICPr/UZs82gAtwYba9NEGaLkZ6qPu+mz6aAO0XA01hMvCmjraAC23vlElLwtD+9w1R1W8LMxzIRb0XaGfUbCLKR++C7HQ7wr9jFoZqs9CzIxpS4haGeqbqguxR09b8Dd+mYIEQoWEu179WZZFZT94t9tFZmZ5nkfr9br/P59X/duq4/Z9/p6+fgHZgbLiMEp7VQAAAABJRU5ErkJggg=="">",0(0.0%)
3,low[float64],Mean (sd) : 140.9 (33.7)min < med < max:53.2 < 145.5 < 197.0IQR (CV) : 45.1 (4.2),"1,003 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACk0lEQVR4nO3cT4riQBiG8a/GVqJCRBp05wUGZuHSQ8xh5wiz8ATewJ0gEjHTwRidzTDQ0KMxf8bvtZ/fPuksHtJlparC5XIxwLsvj34AoIyXRz8A3gsh9M2sV+MWx8vl8tbU83hBqI6EEPqvr6/f4zgeV73Hfr/fhRB+PFushOpLL47j8WKxeBuNRtm9FydJEi2Xy/F2u+2ZGaGiXaPRKJtMJr8qXt5v9GGc4McUJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJLB6Cn95XrRNqDAz/4u2CbUFNd5M8fl87jb9PCW5XrRNqA2r82bK8zwqiuJrlmU/zazqwulavC7aJtTmVX4zrdfr8Wq1GpxOp05bD6eKUFtS5c202+2echtJE5ieggRChQRChQRChQRChQRChQRChQTmUZ9MURRdM4tDCPde+sjPtzcR6hNJ07Sb5/m32WzW6XQ6d30V8/D59hpC/QfFhSXH4/EliqLBfD7PptPp7p5rvX++JdQPqC8sGQ6HT/f5llA/xsISZwj1ChaW+MH0FCQQKiTwrx+NqTGHa3ZjYyChohF15nDNbm8MJFQ0os4cbpmNgYSKRlWZw/3j6mxJq6F6PtAAWloL1fuBBtDS5hvV9YEG0NL6GNXrgQbQwoQ/JBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJLhej9rm1gZocRtq21sboOVmqI862qbtrQ3QcjVUD0fbVN3aUBRFbNWHDa5PtvuMbr1RJY+2qTts8HB+FN4rNUZVO9qmzrDBjPOjPHL7Y6oJVYcNnB/lD/OokECokECokFBqjJokSXTvjQ+HQ2RmlqZptNlsBv/zetW/rfrcda8v09dvL8Ky6X6zR0wAAAAASUVORK5CYII="">",0(0.0%)
4,close[float64],Mean (sd) : 142.6 (33.8)min < med < max:56.1 < 146.8 < 198.1IQR (CV) : 45.3 (4.2),996 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAY

In [59]:
import pandas as pd

df.index = pd.to_datetime(df.index)
df.columns = df.columns.str.lower()
df.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,132079200


In [16]:
!pip install plotly -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 275.7 kB/s eta 0:00:0000:0100:03


In [60]:
import plotly.graph_objects as go

fig = go.Figure([go.Scatter(x=df.index, y=df['close'], mode='lines')])
fig.update_layout(plot_bgcolor='white', 
                  xaxis_title='Date',
                  yaxis_title='Price',
                  title="Apple")
fig.show()

In [74]:
fig = go.Figure([go.Candlestick(x=df.index, 
                                open=df['open'],
                                high=df['high'],
                                low=df['low'],
                                close=df['close'])])
fig.update_layout(xaxis_rangeslider_visible=False,
                  #xaxis=dict(type = "category"),
                  plot_bgcolor='white',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  title="Apple")
fig.update_yaxes(fixedrange=False)
fig.show()

In [73]:
!pip install ccxt -q

In [92]:
import ccxt
import pandas as pd
import time

#current_milli_time = lambda x: int(round((time.time() - 3600 * x) * 1000))

exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1h'
since = exchange.parse8601('2024-01-01T00:00:00Z')
all_ohlcvs = []

while True:
    try:
        ohlcvs = exchange.fetch_ohlcv(symbol, timeframe, since)
        all_ohlcvs += ohlcvs
        if len(ohlcvs):
            print('Fetched', len(ohlcvs), symbol, timeframe, 'candles from', exchange.iso8601(ohlcvs[0][0]))
            since = ohlcvs[-1][0] + 1
            sleep_interval = exchange.rateLimit / 1000
            print('Sleep for', sleep_interval)
            time.sleep(sleep_interval)
        else:
            break
    except Exception as e:
        print(type(e).__name__, str(e))
print('Fetched', len(all_ohlcvs), symbol, timeframe, 'candles in total')
#binance.fetch_ohlcv(symbol, limit=2)

Fetched 500 BTC/USDT 1h candles from 2024-01-01T00:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-01-21T20:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-02-11T16:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-03-03T12:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-03-24T08:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-04-14T04:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-05-05T00:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-05-25T20:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-06-15T16:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-07-06T12:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USDT 1h candles from 2024-07-27T08:00:00.000Z
Sleep for 0.05
Fetched 378 BTC/USDT 1h candles from 2024-08-17T04:00:00.000Z
Sleep for 0.05
Fetched 5878 BTC/USDT 1h candles in total


In [94]:
df = pd.DataFrame(all_ohlcvs)
df.columns = ['date','open','high','low','close','volume']
df = df.sort_values(by='date')
df = df.drop_duplicates(subset='date').reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'], unit='ms')
df

,date,open,high,low,close,volume
0,2024-01-01 00:00:00,42283.58,42554.57,42261.02,42475.23,1271.68108
1,2024-01-01 01:00:00,42475.23,42775.00,42431.65,42613.56,1196.37856
2,2024-01-01 02:00:00,42613.57,42638.41,42500.00,42581.10,685.21980
3,2024-01-01 03:00:00,42581.09,42586.64,42230.08,42330.49,794.80391
4,2024-01-01 04:00:00,42330.50,42399.99,42209.46,42399.99,715.41760
...,...,...,...,...,...,...
5873,2024-09-01 17:00:00,58132.07,58201.00,57864.30,58127.98,571.34425
5874,2024-09-01 18:00:00,58127.98,58788.93,58122.00,58541.10,942.52521
5875,2024-09-01 19:00:00,58541.11,58775.00,58292.00,58514.00,632.24922
5876,2024-09-01 20:00:00,58514.01,58656.77,58381.10,58431.10,334.87245


In [95]:
df.shape

(5878, 6)

In [96]:
df.dtypes

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume           float64
dtype: object

In [99]:
df.isnull().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [100]:
df.describe()

,date,open,high,low,close,volume
count,5878,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000
mean,2024-05-02 10:30:00,60009.273603,60237.626239,59763.907275,60011.987540,1547.353400
min,2024-01-01 00:00:00,38767.750000,38900.000000,38555.000000,38767.740000,99.200980
25%,2024-03-02 05:15:00,56454.392500,56769.390000,56163.825000,56459.862500,653.154627
50%,2024-05-02 10:30:00,62829.715000,63080.555000,62554.480000,62829.720000,1075.767840
75%,2024-07-02 15:45:00,66907.517500,67143.527500,66660.585000,66907.512500,1846.143382
max,2024-09-01 21:00:00,73577.350000,73777.000000,73215.140000,73577.350000,23219.736890
std,NaN,9040.567249,9071.309645,9005.132605,9037.639954,1568.265427


In [101]:
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,date[datetime64[ns]],Min: 2024-01-01Max: 2024-09-01Duration: 244 days,5878 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABNCAYAAACxBha+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACRElEQVR4nO3Yua7aUBRG4X0wIDPIFkKBjuaWkVK45CHysHmEFDwBRZpUdEjIMsL3WhjjFFGkNAyH4SeQ9bWw2QezGISr69oAhcajD4D/B7FBpnnsRudcx8zaFzzutq7rD98h9b5rvPq1uce+g7E55zrD4fBrFEUD323r9Tp1zn3zeZLqfdd49Wtzr33HPtnaURQNptPpRxzHxbnLsiwLZ7PZYLVatc3M58VX77vGq1+bu+w7+jVqZhbHcTEajd49DmpVVUVmFjnnfMai/X7fiuM4Fe0z+/2bde87ZPqzPs0+M+scuuFkbL7yPG+VZfllMpkEQRCc/a4oyzKsqupzURTfzezsJ3jpvqqqWnmev/X7/Z+NRqM8d+4RZ32WfafcPLbtdtsMw7CbJEkxHo/Tc+cWi8VgPp93d7tdINz3KUmSHz5zDzzrP7/vlJvH9kev1/P6+k3T9ODH7z33+c79PXupZ7k2t8b/bJAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbINE/dIcuy0OcBN5tNaGaW53m4XC67rzb3TGdVz51q5RcE36ma7cH2JAAAAABJRU5ErkJggg=="">",0(0.0%)
2,open[float64],Mean (sd) : 60009.3 (9040.6)min < med < max:38767.8 < 62829.7 < 73577.4IQR (CV) : 10453.1 (6.6),"5,796 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACqElEQVR4nO3cwW6iUBSA4cNYlaqBGBPd+QKTzMIlDzEPO4/QhU/gG7gzMQYjLRGVWcw0mUVHhSvlnNv/20MJ+XPjpQeCsiwF0O5b2xcA3OOp7QuAHkEQPItIz+EUx7Is3x51Pf8iVIjIn0gnk8nPKIrGdc+x3+93QRD8aiJWQvWMw6oYjUajaZIk+ziO86oHp2kaLpfL8Xa77YkIoeL/XFbFoijC8/n8vd/vv0yn09eal/Bc87ibCNUvvSiKxkmSvFVdFdfr9Xi1Wg1Op1OnqYtzQageiuM4r7oq7na7xlbDR+DxFEwgVJhAqDCBUGECocIEQoUJhAoTCBUmECpM4D9TyjiO2kWXy6X7yOvRglAVcR21ex8syfP8RUTqDpaoRKi61B4qEdE/WOKCUBWqM1Qion+wxAWbKZhAqDCBUGECocIEQoUJhAoTCBUmECpMIFSYQKgwgVBhAqHCBEKFCYQKExjza4DLpx99ndB3RagP9ohPP/o4oe+q0VA1f2q7Qd5++rFNjYWq/VPbTfPx049tanJFdXr/p+lPbcOWxn+j1n3/5y9WGIgIj6dgBKHCBEKFCYQKEwgVJhAqTCBUmECoMIFQYQKhwgRvx/y+6ORWq87nc1dEoiAI6hx+9X57GepXn9xqQ5Zl3aIofszn806n06k8hHTrfnsZqjC59emOx+NTGIaDxWKRz2azXZVj77nfvoYqIkxutWE4HNa951fv981Qef8HGlwNlfd/oMWtFZX3f6DCXb9Ref8HbeOBP0xQvet3eIDMRs4zakN1eYDMRs4/akN1eYDMRs4/akN9V+cBMhs5/7CZggnqV9S2sJHThVA/wEZOH0L9ABs5fQj1CjZyetwVapqmYdUTHw6HUEQky7Jws9kMPvN4q3/b6nW7Hn9PX78Bea2hE2CSqv4AAAAASUVORK5CYII="">",0(0.0%)
3,high[float64],Mean (sd) : 60237.6 (9071.3)min < med < max:38900.0 < 63080.6 < 73777.0IQR (CV) : 10374.1 (6.6),"5,634 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACqUlEQVR4nO3cMY7aQBSA4eewLF5ARggJOi4QKQWlD5HD5ghbcAJuQIeEkBHOWhjjFMlKKTaAPXb83uz/9YYpfo0Y/OygLEsBtPvS9QKARzx1vQDoEQTBi4g8O3zEuSzLt6bW8zdChYj8jnQ2m32Pomha9zOOx+MhCIIfbcRKqJ5x2BWj8Xg8j+P4OJlMsqoXJ0kSrtfr6X6/fxYRQsW/ueyKeZ6HRVF8HQwGr/P5/GfNJbzUvO4uQvXLcxRF0ziO36ruitvtdrrZbIaXy6XX1uJcEKqHJpNJVnVXPBwOre2GTeDvKZhAqDCBUGECocIEQoUJhAoTCBUmECpMIFSYQKgwgVBhAvf6lXEcXo6u12u/yfVoQaiKuA4vv4/qZVn2KiJ1R/VUIlRdao/piegf1XNBqArVGdMT0T+q54LDFEwgVJhAqDCBUGECocIEQoUJhAoTCBUmECpMIFSYQKgwgVBhAqHCBKanWuDyjlJfB59dEWrDmnhHqY+Dz65aDVXzO+Fb5O07SrvUWqja3wnfNh/fUdqlNndUp8cq2n4nPGxp/Tdq3ccq/mCHgYjw9xSMIFSYQKgwgVBhAqHCBEKFCYQKEwgVJjCUgsYURdEXkSgIgjqX35zr8DbUTzoQ05k0Tft5nn9bLpe9Xq9X+Zb5vbkOL0P97AMxXTifz09hGA5Xq1W2WCwOVa59ZK7Dy1CFgZjOjEajurMdN+c6fA1VRBiI8cndUHmsAhrcDJXHKqDFvR2VxyqgwkO/UXmsAl3jzhRMUH3qd7jTwUHOM2pDdbnTwUHOP2pDdbnTwUHOP2pDfVfnTgcHOf9wmIIJ6nfUrnCQ04VQP8BBTh9C/QAHOX0I9QYOcno8FGqSJGHVDz6dTqGISJqm4W63G/7P661+t9V1u17/SF+/APlXoRWuGtvOAAAAAElFTkSuQmCC"">",0(0.0%)
4,low[float64],Mean (sd) : 59763.9 (9005.1)min < med < max:38555.0 < 62554.5 < 73215.1IQR (CV) : 10496.8 (6.6),"5,651 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXR

In [ ]:
df['date'].diff().value_counts()

In [103]:
import plotly.graph_objects as go

fig = go.Figure([go.Scatter(x=df['date'], y=df['close'], mode='lines')])
fig.update_layout(plot_bgcolor='white', 
                  xaxis_title='Date',
                  yaxis_title='Price',
                  title="BTC/USDT")
fig.show()

In [105]:
fig = go.Figure([go.Candlestick(x=df['date'], 
                                open=df['open'],
                                high=df['high'],
                                low=df['low'],
                                close=df['close'])])
fig.update_layout(xaxis_rangeslider_visible=False,
                  #xaxis=dict(type = "category"),
                  plot_bgcolor='white',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  title="BTC/USDT")
#fig.update_yaxes(fixedrange=False)
fig.show()

In [106]:
df['date'].diff().value_counts()

date
0 days 01:00:00    5877
Name: count, dtype: int64

In [84]:
#data = binance.fetch_ohlcv(pair, '1d')
#data

[[1682035200000, 28243.65, 28374.02, 27125.0, 27262.84, 77684.7679],
 [1682121600000, 27262.84, 27882.72, 27140.35, 27816.85, 36023.69686],
 [1682208000000, 27816.85, 27816.85, 27311.25, 27590.6, 34812.09581],
 [1682294400000, 27590.59, 28000.0, 26942.82, 27510.93, 53111.56874],
 [1682380800000, 27510.93, 28399.99, 27192.0, 28300.79, 52325.14637],
 [1682467200000, 28300.8, 30036.0, 27235.0, 28415.29, 129228.40403],
 [1682553600000, 28415.29, 29890.0, 28378.86, 29472.77, 95430.82431],
 [1682640000000, 29472.77, 29599.54, 28891.0, 29311.7, 54298.16578],
 [1682726400000, 29311.69, 29448.88, 29031.0, 29230.45, 20466.83058],
 [1682812800000, 29230.45, 29969.39, 29079.59, 29233.21, 39752.5372],
 [1682899200000, 29233.2, 29337.34, 27666.95, 28068.26, 64433.65958],
 [1682985600000, 28068.26, 28879.88, 27872.0, 28669.86, 50824.5224],
 [1683072000000, 28669.85, 29266.66, 28113.69, 29026.16, 64615.79213],
 [1683158400000, 29026.16, 29379.83, 28663.64, 28838.16, 42575.47501],
 [1683244800000, 2883

In [88]:
#import pandas as pd
#df = pd.DataFrame()

#Получим данные за 6 месяцев назад в таймфрейме по часу
#for hours in range(4320,0,-600): # 6 месяцев = 24 часа * 30 дней * 6 = 4320
# for hours in range(24,0,-1): # 6 месяцев = 24 часа * 30 дней * 6 = 4320
#     if binance.has['fetchOHLCV']:
#         time.sleep(binance.rateLimit / 1000)
#         data = binance.fetch_ohlcv(pair, '1h', since=current_milli_time(hours),
#                                     limit=1000)
#         df = pd.concat([df, pd.DataFrame(data)])
#         print(hours)
# df.head()

24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


,0,1,2,3,4,5
0,1725141600000,59061.00,59141.72,59006.00,59024.00,203.35984
1,1725145200000,59023.99,59056.00,58927.00,58973.99,283.56516
2,1725148800000,58974.00,59076.59,58900.00,58930.00,289.12701
3,1725152400000,58929.99,58957.00,58819.78,58828.01,330.21542
4,1725156000000,58828.00,58851.65,58276.01,58540.25,1256.77642
